<a href="https://colab.research.google.com/github/little-hoge/PythonAI/blob/main/2%E6%97%A5%E7%9B%AE/test4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# ５章
---

---
#### ファイルパス設定
---

In [1]:
# ファイル読み込み
from google.colab import drive 
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/2日目"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/2日目


---
#### 前処理
---

In [2]:
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection

#import warnings
#warnings.filterwarnings('ignore')

classes = ["monkey","boar","crow"]
num_classes = len(classes)
image_size = 50

# 画像の読み込み
X = []
Y = []
for index, classlabel in enumerate(classes):
  
    #フォルダを指定
    photos_dir = "animalai/" + classlabel
    #ファイルを指定
    files = glob.glob(photos_dir + "/*.jpg")
    print(files)
    for i, file in enumerate(files):
        if i >= 200: break
        #ファイルを開く
        image = Image.open(file)
        #RGBそれぞれ00～FF（256色）で表しているため、
        #1ドットづつ数値化し正解ラベルとともに数値化
        image = image.convert("RGB")
        #50×50のサイズにリサイズ
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        X.append(data)
        Y.append(index)

X = np.array(X).astype("float") / 256
Y = np.array(Y)
xy = model_selection.train_test_split(X, Y, test_size=0.3, random_state=0)

#numpyの行列ファイルとして保存
np.save("./animal.npy", xy)

['animalai/monkey/68347317.jpg', 'animalai/monkey/214759783.jpg', 'animalai/monkey/5610331.jpg', 'animalai/monkey/19426192.jpg', 'animalai/monkey/297887549.jpg', 'animalai/monkey/115828444.jpg', 'animalai/monkey/203757258.jpg', 'animalai/monkey/25255858.jpg', 'animalai/monkey/249719619.jpg', 'animalai/monkey/10353311.jpg']
['animalai/boar/212327098.jpg', 'animalai/boar/36025158.jpg', 'animalai/boar/190619675.jpg', 'animalai/boar/156766490.jpg', 'animalai/boar/74627946.jpg', 'animalai/boar/247083615.jpg', 'animalai/boar/75968726.jpg', 'animalai/boar/43136063.jpg', 'animalai/boar/64635188.jpg', 'animalai/boar/107658675.jpg']
['animalai/crow/8409993.jpg', 'animalai/crow/42193260.jpg', 'animalai/crow/33681381.jpg', 'animalai/crow/3574240.jpg', 'animalai/crow/36880264.jpg', 'animalai/crow/26355585.jpg', 'animalai/crow/7088908.jpg', 'animalai/crow/269039.jpg', 'animalai/crow/2738037.jpg', 'animalai/crow/11163861.jpg']


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


---
#### ライブラリのインポート
---

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import from_numpy, manual_seed
from torch.utils.data import DataLoader, TensorDataset

---
#### 行列の形状を確認し、変形
---

In [4]:
X_train, X_test, y_train, y_test = np.load("./animal.npy" , allow_pickle=True)

print("変形前:", X_train.shape)

# 形状を（件数、縦、横、RGB）から（件数、RGB、縦、横）に変換する
X_train = np.transpose(X_train, (0, 3, 1, 2))
X_test = np.transpose(X_test, (0, 3, 1, 2))

print("変更後:", X_train.shape)

変形前: (21, 50, 50, 3)
変更後: (21, 3, 50, 50)


---
#### テンソル型に変換し、データローダにセット
---

In [5]:
#numpyのデータをテンソル型に変換
X_train = from_numpy(X_train.astype(np.float32))
y_train = from_numpy(y_train.astype(np.int64))
X_test = from_numpy(X_test.astype(np.float32))
y_test = from_numpy(y_test.astype(np.int64))

#テンソル型のXとYをデータローダーにセット
train = TensorDataset(X_train, y_train)

# PyTorch のRNGシードを初期化（再現性の為）
manual_seed(0)

#15件づつに分割してシャッフル
train_loader = DataLoader(train, batch_size=15, shuffle=True)

---
#### 学習モデルを定義
---

In [6]:
# CNNを実装する
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

---
#### 機械学習実行
---

In [7]:
model = Net()
# 交差エントロピー
criterion = nn.CrossEntropyLoss()
# 確率的勾配降下法
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(500): #学習回数500回
    total_loss = 0

    for X_train, y_train in train_loader:
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    #途中の損失関数を確認
    if (epoch + 1) % 50 == 0:
        print(epoch + 1, total_loss)

50 1.633530080318451
100 0.0006062689499231055
150 0.0002677513512026053
200 0.00013577770732808858
250 0.00010341083907405846
300 8.235440327553079e-05
350 7.883145553932991e-05
400 4.05128565432733e-05
450 5.484143730427604e-05
500 5.173037607164588e-05


---
#### 学習済みモデルの保存
---

In [8]:
# モデルを保存する
PATH = './animal_cnn.pth'
torch.save(model.state_dict(), PATH)

---
#### ライブラリと前処理済み画像データの読み込み
---

In [9]:

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import from_numpy, manual_seed
from torch.utils.data import DataLoader, TensorDataset

# 前処理済み画像データの読み込み
X_train, X_test, y_train, y_test = np.load("./animal.npy", allow_pickle=True)

# 形状を（件数、縦、横、RGB）から（件数、RGB、縦、横）に変換する
X_train = np.transpose(X_train, (0, 3, 1, 2))
X_test = np.transpose(X_test, (0, 3, 1, 2))

#numpyのデータをテンソル型に変換
X_train = from_numpy(X_train.astype(np.float32))
y_train = from_numpy(y_train.astype(np.int64))
X_test = from_numpy(X_test.astype(np.float32))
y_test = from_numpy(y_test.astype(np.int64))

---
#### 学習モデルを定義
---

In [10]:
# CNNを実装する
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

---
#### 学習済みモデルを読み込み
---

In [11]:
model = Net()
PATH = './animal_cnn.pth'
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

---
#### 精度確認
---

In [12]:
#テストデータで予測結果を作成
y_prob , y_pred = torch.max(model(X_test).data, 1)

accuracy = accuracy_score(y_test, y_pred)

print("正確度：",accuracy)

# 正確度（正解率）を算出　(TP + TN) / 全体
accuracy = accuracy_score(y_test, y_pred)

# 適合度を算出　TP / (TP + FP)
precision = precision_score(y_test, y_pred, average=None)

#再現度を算出　TP / (TP + FN)
recall = recall_score(y_test, y_pred, average=None)

#F1スコアを算出 (2 - 適合度 × 再現度) / (適合度 × 再現度)
f1 = f1_score(y_test, y_pred, average=None)

print("正確度（Accuracy）                   適合度（Precision） ") 
print(accuracy, "                 ", precision) 
print("再現度（Recall）                     F1スコア（F1） ") 
print(recall, " ", f1) 

正確度： 0.2222222222222222
正確度（Accuracy）                   適合度（Precision） 
0.2222222222222222                   [0.         0.66666667 0.        ]
再現度（Recall）                     F1スコア（F1） 
[0.  0.5 0. ]   [0.         0.57142857 0.        ]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
